In [12]:
import pandas as pd
import pickle
import numpy as np
import shapefile
from scipy.spatial import cKDTree
from os import listdir
from os.path import isfile, join
from sklearn.decomposition import TruncatedSVD

### DataFrame

In [124]:
df = pd.read_csv('./input/train.csv')

noAgg = [c for c in df.columns if c not in ['NumMosquitos','WnvPresent']]

agg = df.groupby(noAgg)['NumMosquitos', 'WnvPresent'].sum()

for i, c in enumerate(noAgg):
    agg[c] = agg.index.map(lambda x:x[i])

agg.index = range(0,len(agg))

agg['WnvPresent'] = (agg['WnvPresent'].map(lambda x:x>0)).astype(int)

df = agg

def cleanX(xdata):
    species = ['CULEX PIPIENS', 'CULEX PIPIENS/RESTUANS',
                'CULEX RESTUANS', 'CULEX SALINARIUS',
                'CULEX TERRITANS', 'CULEX TARSALIS',
                 'CULEX ERRATICUS']
    
    toDrop = [
        'Address','Block','Street',
              'AddressNumberAndStreet', 'AddressAccuracy',
             ]
    
    toRet = xdata.copy()
    toRet['Date'] = pd.to_datetime(toRet['Date'])
    
    toRet['Yr'] = [t.year for t in toRet['Date']]
    toRet['Mo'] = [t.month for t in toRet['Date']]
    #toRet['Day'] = [t.day for t in toRet['Date']]
    toRet['Week'] = [t.week for t in toRet['Date']]
    
    toRet['Location'] = [(toRet.loc[idx,'Longitude'], toRet.loc[idx,'Latitude'])
                  for idx in toRet.index]
    for s in species:
        toRet[s] = (toRet['Species'] == s).astype(int)
        
    toRet = toRet.drop(toDrop, axis = 'columns')
    
    return toRet

df = cleanX(df)

df.head(2)
df.shape

(8475, 18)

In [118]:
v=df['Trap'].iloc[0]

### Weather

In [14]:
weather = pd.read_csv('./input/weather.csv')

#weather info: years: 2007-2014 months: 5,6,7,8,9,10, 1 observation per day per station

#Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
#Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level


#Convert datatypes
weather['Date'] = pd.to_datetime(weather['Date'])

weather['Yr'] = weather['Date'].dt.year
weather['Mo'] = weather['Date'].dt.month
weather['Day'] = weather['Date'].dt.day

#Drop columns that we are not interested in 
weather = weather.drop({'Depart', 'Depth', 'Water1', 'SnowFall', 'CodeSum', 'Heat', 'Cool'}, axis=1)

#Replace 'M' = missing value with NaN & '  T' with NaN
weather = weather.replace('M', np.nan)
#Replace'  T' with 0.001 
weather = weather.replace('  T', 0.001)
#Replace'  -' with 0000 
weather = weather.replace('-', '0000')

#Convert datatypes
weather['Tavg'] = weather['Tavg'].astype(float)
weather['WetBulb'] = weather['WetBulb'].astype(float)

weather['Sunrise'] = pd.to_datetime(weather['Sunrise'], format="%H%M")

#Fix some data ex: 16:60 --> 1700
weather['Sunset'] = [date if date[-2:] != '60' else str(int(date[0:2])+1)+'00' for date in weather['Sunset']]
weather['Sunset'] = pd.to_datetime(weather['Sunset'], format="%H%M")

weather['PrecipTotal'] = weather['PrecipTotal'].astype(float)
weather['StnPressure'] = weather['StnPressure'].astype(float)
weather['SeaLevel'] = weather['SeaLevel'].astype(float)
weather['ResultSpeed'] = weather['ResultSpeed'].astype(float)
weather['AvgSpeed'] = weather['AvgSpeed'].astype(float)

#Drop null values 
weather.dropna(inplace=True)

## At this point, Weather is in good shape
#Weather from Station 1 
weather_st1 = weather[weather['Station']== 1]

#Weather from Station 2
weather_st2 = weather[weather['Station']== 2]

In [37]:
df.head(2)

,NumMosquitos,WnvPresent,Date,Species,Trap,Latitude,Longitude,Yr,Mo,Week,Location,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TERRITANS,CULEX TARSALIS,CULEX ERRATICUS
0,1,0,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,2007,5,22,"(-87.654224, 41.867108)",0,1,0,0,0,0,0
1,2,0,2007-05-29,CULEX RESTUANS,T048,41.867108,-87.654224,2007,5,22,"(-87.654224, 41.867108)",0,0,1,0,0,0,0


In [107]:
pd.Series([2,4])

0    2
1    4
dtype: int64

In [ ]:
plt.plot(df['Week'], lr.intercept_ )

In [114]:
df.head()

,Tavg,Week,Week^2
Week,,,
17,60.833333,0,0
18,56.740741,1,1
19,59.625000,2,4
20,60.535714,3,9
21,64.803571,4,16


In [112]:
lr.coef_

array([ 3.28641226, -0.13246797])

In [122]:
weather['Week'] = weather['Date'].dt.week
wtw = weather.groupby('Week')['Tavg'].mean()
df = pd.DataFrame(wtw)

df['Week'] = df.index -17
df['Week^2'] = df['Week']**2

lr = LinearRegression()
lr.fit(df.drop('Tavg', axis = 'columns'), df['Tavg'])
toRet = {}
for w in range(1,53):
    toRet[w] = lr.intercept_ + (lr.coef_[0]*(w-17)) + (lr.coef_[1] * ((w-17)**2))
toRet
atd = toRet

In [153]:

def calculate_agregate( weather_sub, avgTDict):
    toRet = pd.Series()

    allAgg = [np.max, np.min, np.mean]
    toAgg = {'DewPoint': allAgg,
            'StnPressure': allAgg,
            'AvgSpeed': allAgg,
            'Tmax':[np.max],
            'Tmin':[np.min],
            'Tavg':[np.mean],
            'PrecipTotal':[np.sum, np.mean]
            }
    for k in toAgg:
        for f in toAgg[k]:
            toRet.loc[k + str(f).split(' ')[1]] = f(weather_sub[k])

    finalEntry = weather_sub.iloc[len(weather_sub)-1]

    toRet['temp_expected'] = avgTDict[pd.to_datetime(finalEntry['Date']).week]
    toRet['temp_diff'] = toRet['Tavgmean'] - toRet['temp_expected']

    sunset = finalEntry['Sunset']
    toRet['sunset'] = sunset.hour + (sunset.minute / 60)

    return toRet

def date_ranges(dates):
    uniqueYears = set([pd.to_datetime(d).year for d in dates])

    dates = sorted(dates)
    fyear = []
    for y in uniqueYears:
        for d in dates:
            if pd.to_datetime(d).year == y:
                fyear.append(d)
                break

    for d in fyear:
        dates = np.insert(dates, 0, d - pd.Timedelta(days = 8))

    dateRanges = []
    for i in range(len(dates)-1):
        if pd.to_datetime(dates[i]).year == pd.to_datetime(dates[i+1]).year:
            dateRanges.append( (dates[i], dates[i+1]) )

    return dateRanges

def subset_weather(dateRange, weather):
    mask = (weather['Date']>dateRange[0]) & (weather['Date'] <= dateRange[1])
    return weather.loc[mask]

def TWeatherDFMaker(dct):
    toRet = pd.DataFrame().from_dict(dct)
    toRet = toRet.transpose()
    toRet.index = [idx for idx in toRet.index]
    toRet['Trap'] = toRet.index.map(lambda x: x[0])
    toRet['Date'] = toRet.index.map(lambda x: x[1])
    toRet.index = range(len(toRet))

    return toRet

def trap_agregator(trap_df, weather, avgTDict):
    trapWeather = {}
    trap = trap_df['Trap'].iloc[0]

    dates = trap_df['Date'].unique()
    dates = sorted(dates)

    dateRanges = date_ranges(dates)

    for dr in dateRanges:
        weather_sub = subset_weather(dr, weather)
        trapWeather[(trap, dr[1])] = calculate_agregate(weather_sub, avgTDict)
    toRet = pd.DataFrame().from_dict(trapWeather)

    return TWeatherDFMaker(trapWeather)


In [154]:
tr = trap_agregator(t48, weather_st1, atd)

In [157]:
pd.concat([tr,tr],axis = 'rows')

,DewPointamax,DewPointamin,DewPointmean,StnPressureamax,StnPressureamin,StnPressuremean,AvgSpeedamax,AvgSpeedamin,AvgSpeedmean,Tmaxamax,Tminamin,Tavgmean,PrecipTotalsum,PrecipTotalmean,temp_expected,temp_diff,sunset,Trap,Date
0,63.0,48.0,58.714286,29.31,28.82,29.078571,10.6,6.4,8.328571,87.0,47.0,69.571429,1.060,0.151429,69.964018,-0.392590,19.383333,T048,2007-06-05
1,69.0,47.0,56.142857,29.44,28.84,29.275714,23.1,4.9,9.171429,93.0,44.0,72.952381,1.092,0.052000,73.862196,-0.909815,19.516667,T048,2007-06-26
2,70.0,46.0,54.666667,29.51,29.29,29.420000,12.5,5.8,8.683333,91.0,53.0,68.000000,0.150,0.025000,74.631717,-6.631717,19.516667,T048,2007-07-02
3,67.0,51.0,61.444444,29.32,29.05,29.186667,13.1,5.5,9.000000,94.0,60.0,77.555556,1.290,0.143333,75.136302,2.419253,19.466667,T048,2007-07-11
4,68.0,51.0,57.857143,29.26,29.08,29.184286,13.2,4.9,8.042857,85.0,58.0,72.857143,2.050,0.292857,75.375951,-2.518808,19.400000,T048,2007-07-18
5,68.0,52.0,58.888889,29.51,29.11,29.325556,11.9,3.3,7.200000,85.0,58.0,72.444444,0.521,0.057889,75.350664,-2.906220,19.283333,T048,2007-07-27
6,65.0,55.0,60.000000,29.33,29.23,29.292000,10.2,4.1,5.840000,92.0,62.0,76.800000,0.000,0.000000,75.060441,1.739559,19.183333,T048,2007-08-01
7,73.0,56.0,66.000000,29.31,29.06,29.191667,9.6,4.9,6.866667,91.0,64.0,79.166667,3.030,0.505000,74.505282,4.661384,19.066667,T048,2007-08-07
8,71.0,57.0,66.625000,29.39,29.13,29.245000,8.0,3.9,6.100000,89.0,65.0,77.000000,1.040,0.130000,73.685188,3.314812,18.900000,T048,2007-08-15
9,70.0,54.0,62.000000,29.44,29.15,29.288333,8.1,5.7,6.616667,87.0,59.0,70.166667,2.331,0.388500,72.600157,-2.433490,18.750000,T048,2007-08-21


In [148]:
t2 = tr
t2 = t2.transpose()
t2.index = [idx for idx in t2.index]
t2.head()

,DewPointamax,DewPointamin,DewPointmean,StnPressureamax,StnPressureamin,StnPressuremean,AvgSpeedamax,AvgSpeedamin,AvgSpeedmean,Tmaxamax,Tminamin,Tavgmean,PrecipTotalsum,PrecipTotalmean,temp_expected,temp_diff,sunset
"(T048, 2007-06-05 00:00:00)",63.0,48.0,58.714286,29.31,28.82,29.078571,10.6,6.4,8.328571,87.0,47.0,69.571429,1.060,0.151429,69.964018,-0.392590,19.383333
"(T048, 2007-06-26 00:00:00)",69.0,47.0,56.142857,29.44,28.84,29.275714,23.1,4.9,9.171429,93.0,44.0,72.952381,1.092,0.052000,73.862196,-0.909815,19.516667
"(T048, 2007-07-02 00:00:00)",70.0,46.0,54.666667,29.51,29.29,29.420000,12.5,5.8,8.683333,91.0,53.0,68.000000,0.150,0.025000,74.631717,-6.631717,19.516667
"(T048, 2007-07-11 00:00:00)",67.0,51.0,61.444444,29.32,29.05,29.186667,13.1,5.5,9.000000,94.0,60.0,77.555556,1.290,0.143333,75.136302,2.419253,19.466667
"(T048, 2007-07-18 00:00:00)",68.0,51.0,57.857143,29.26,29.08,29.184286,13.2,4.9,8.042857,85.0,58.0,72.857143,2.050,0.292857,75.375951,-2.518808,19.400000


In [127]:
t48= df[df['Trap']=='T048']
t48.head()

,NumMosquitos,WnvPresent,Date,Species,Trap,Latitude,Longitude,Yr,Mo,Week,Location,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TERRITANS,CULEX TARSALIS,CULEX ERRATICUS
0,1,0,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,2007,5,22,"(-87.654224, 41.867108)",0,1,0,0,0,0,0
1,2,0,2007-05-29,CULEX RESTUANS,T048,41.867108,-87.654224,2007,5,22,"(-87.654224, 41.867108)",0,0,1,0,0,0,0
25,2,0,2007-06-05,CULEX PIPIENS,T048,41.867108,-87.654224,2007,6,23,"(-87.654224, 41.867108)",1,0,0,0,0,0,0
26,1,0,2007-06-05,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,2007,6,23,"(-87.654224, 41.867108)",0,1,0,0,0,0,0
27,1,0,2007-06-05,CULEX RESTUANS,T048,41.867108,-87.654224,2007,6,23,"(-87.654224, 41.867108)",0,0,1,0,0,0,0


In [53]:
from sklearn.linear_model import LinearRegression

def aggWeatherForObs(data, weather):
    agg = []

    weather['Week'] = weather['Date'].dt.week
    wtw = weather.groupby('Week')['Tavg'].mean()
    df = pd.DataFrame(wtw)
    
    df['Week'] = df.index -17
    df['Week^2'] = df['Week']**2
    
    lr = LinearRegression()
    lr.fit(df.drop('Tavg', axis = 'columns'), df['Tavg'])
    
    def weeklyAvgTemp(weeknum, lm):
        wk = weeknum - 17
        
        return lm.intercept_ + (lr.coef_[0]*wk) + (lr.coef_[1]* (wk**2))
    
    def calculate_agregate (trap_df, weather_sub, end_date):

        toRet = pd.Series()

        toRet['Trap'] = trap_df.iloc[0,4] 
        toRet['Date_end'] = pd.to_datetime(end_date)
        
        for c in ['DewPoint', 'StnPressure', 'AvgSpeed']:
            for f in [np.max, np.min, np.mean]:
                toRet.loc[c+str(f).split(' ')[1]] = f(weather_sub[c])

        toRet['temp_max'] = weather_sub['Tmax'].max()
        toRet['temp_min'] = weather_sub['Tmin'].min()
        toRet['temp_avg'] = weather_sub['Tavg'].mean()
        toRet['temp_expected'] = weeklyAvgTemp(pd.to_datetime(toRet['Date_end']).week , lr)
        toRet['temp_diff'] = toRet['temp_avg'] - toRet['temp_expected']
        
        #print(weather_sub['Date'])
        
        try:
            sunset = weather_sub.iloc[len(weather_sub)-1]['Sunset']
        except:
            print('no')
            
        toRet['sunset'] = sunset.hour + (sunset.minute / 60)
        
        toRet['precip_total'] = weather_sub['PrecipTotal'].sum()
        toRet['precip_avg'] = toRet['precip_total'] / len(weather_sub)
        
        return toRet
    
    trapDFs = []
    for trap in data['Trap'].unique():
        trapDFs.append(data[ data['Trap'] == trap ])

    for tDF in trapDFs:

        dates = tDF['Date'].unique()
        dates = sorted(dates)
        
        fyear= {}
        
        for y in [2007, 2009, 2011, 2013]: # needs to be generalized
            for i, d in enumerate(dates):
                if pd.to_datetime(d).year == y:
                    fyear[i] = d
                    break
                           
        for idx in fyear:
            dates = np.insert(dates, 0, fyear[idx] - pd.Timedelta(days = 8))
        dates = sorted(dates)
        
        dateRanges = []
        for i in range(len(dates)-1):
            if pd.to_datetime(dates[i]).year == pd.to_datetime(dates[i+1]).year:
                dateRanges.append( (dates[i] , dates[i+1]) )

        for dr in dateRanges:
            start_date = dr[0]
            end_date = dr[1]

            #Select dataframe rows between two dates in weather
            mask = (weather['Date'] > start_date) & (weather['Date'] <= end_date)
            
            weather_sub = weather.loc[mask]
            
            agg.append( calculate_agregate(tDF, weather_sub, end_date) )
            
    toRet = pd.DataFrame(agg)
    toRet['Date'] = [pd.to_datetime(d) for d in toRet['Date_end']]
    toRet = toRet.drop('Date_end', axis = 'columns')
    return toRet

In [36]:
weatherObs.head()

,Trap,DewPointamax,DewPointamin,DewPointmean,StnPressureamax,StnPressureamin,StnPressuremean,AvgSpeedamax,AvgSpeedamin,AvgSpeedmean,temp_max,temp_min,temp_avg,temp_expected,temp_diff,sunset,precip_total,precip_avg,Date
0,CULEX PIPIENS/RESTUANS,58,44,50.000000,29.51,29.34,29.396250,17.3,5.3,9.575000,89,49,68.375000,67.484959,0.890041,19.283333,1.072,0.134000,2007-05-29
1,CULEX PIPIENS/RESTUANS,63,48,58.714286,29.31,28.82,29.078571,10.6,6.4,8.328571,87,47,69.571429,69.319181,0.252248,19.383333,1.060,0.151429,2007-06-05
2,CULEX PIPIENS/RESTUANS,69,47,56.142857,29.44,28.84,29.275714,23.1,4.9,9.171429,93,44,72.952381,73.228946,-0.276565,19.516667,1.092,0.052000,2007-06-26
3,CULEX PIPIENS/RESTUANS,70,46,54.666667,29.51,29.29,29.420000,12.5,5.8,8.683333,91,53,68.000000,74.001234,-6.001234,19.516667,0.150,0.025000,2007-07-02
4,CULEX PIPIENS/RESTUANS,67,51,61.444444,29.32,29.05,29.186667,13.1,5.5,9.000000,94,60,77.555556,74.508038,3.047517,19.466667,1.290,0.143333,2007-07-11


In [39]:
%%time
weatherObs = aggWeatherForObs(df, weather_st1)
df_w = df.merge( weatherObs, left_on=['Date', 'Trap'],
              right_on=['Date', 'Trap'],
              how='outer')

C:\Users\William\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Wall time: 1min 9s


### Parks/Water

In [56]:
df_w.head(2)

,NumMosquitos,WnvPresent,Date,Species,Trap,Latitude,Longitude,Yr,Mo,Week,...,AvgSpeedamin,AvgSpeedmean,temp_max,temp_min,temp_avg,temp_expected,temp_diff,sunset,precip_total,precip_avg
0,1,0,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,2007,5,22,...,5.3,9.575,89,49,68.375,67.484959,0.890041,19.283333,1.072,0.134
1,2,0,2007-05-29,CULEX RESTUANS,T048,41.867108,-87.654224,2007,5,22,...,5.3,9.575,89,49,68.375,67.484959,0.890041,19.283333,1.072,0.134


In [311]:
parkDir = './AddData/Parks/'
waterDir = './AddData/Water/'

uniqueLocs = df['Location'].unique()


def buildWaterAndParkDicts(parkDir, waterDir):

    # Park and water shapefile names
    parks = [f for f in listdir(parkDir) if isfile(join(parkDir,f)) if f.count('.csv') ==0]
    water = [f for f in listdir(waterDir) if isfile(join(waterDir,f))if f.count('.csv') ==0]

    parkShape = parkDir + parks[0].split('.')[0]
    waterShape = waterDir + water[0].split('.')[0]

    # Read in shapefiles, then the shapes/records
    psf = shapefile.Reader(parkShape)
    wsf = shapefile.Reader(waterShape)

    parkSR = psf.shapeRecords()
    waterSR = wsf.shapeRecords()

    # Create a parksize dictionary, keyed on parkname
    parkSize = {}
    for s in parkSR:
        parkSize[s.record[4]] = s.record[19]

    # Create cKDTree functions in Dict

    # Key: Identifier (number for water, park name for park)
    # Value: cKDTree function built on all the points associated with water/park feature

    waterFinder={}
    for i, s in enumerate(waterSR):
        waterFinder[i] = cKDTree(s.shape.points)

    parkFinder = {}
    for s in parkSR:
        parkFinder[s.record[4]] = cKDTree(s.shape.points)
    
    return parkSize, parkFinder, waterFinder

def yeildParkSVD(parkSize, parkFinder, uniqueLocs, TruncSVD = 'calc', comps = 4):
    
    parkDist = {}
    
    for l in uniqueLocs:
        parkDist[l] = {}
        for k in parkFinder:
            dist = parkFinder[k].query(l,1)[0]
            size = parkSize[k]
            parkDist[l][k] = (dist, size, size/(dist**2))
    
    parkDF = pd.DataFrame()
    parkDF = parkDF.from_dict(parkDist)
    parkDF = parkDF.transpose()
    parkDF.index = [idx for idx in parkDF.index]
    
    for c in parkDF:
        #print(type(parkDF.iloc[0][c]))
        parkDF[c+' Area'] = [e[1] for e in parkDF[c]]
        parkDF[c+ ' Effect'] = [e[2] for e in parkDF[c]]
        parkDF[c] = [e[0] for e in parkDF[c]]
    
    if TruncSVD == 'calc':
        TruncSVD = TruncatedSVD(n_components = comps)
        TruncSVD.fit(parkDF)
    
    toRet = TruncSVD.transform(parkDF)
    
    toRet = pd.DataFrame(toRet, index = parkDF.index)
    return toRet, TruncSVD

def yeildWaterSVD(waterFinder, uniqueLocs, TruncSVD = 'calc', comps = 4):
    
    waterDist = {}
    
    for l in uniqueLocs:
        waterDist[l] = {}
        for k in waterFinder:
            waterDist[l][k] = waterFinder[k].query(l,1)[0]
    
    waterDF = pd.DataFrame()
    waterDF = waterDF.from_dict(waterDist)
    waterDF = waterDF.transpose()
    waterDF.index = [idx for idx in waterDF.index]
    print(type(waterDF.iloc[0,0])) 
    if TruncSVD == 'calc':
        TruncSVD = TruncatedSVD(n_components = comps)
        TruncSVD.fit(waterDF)
    print(waterDF.shape)
    print(TruncSVD)
    toRet = TruncSVD.transform(waterDF)
    
    toRet = pd.DataFrame(toRet, index = waterDF.index)
    print(toRet.shape)
    return toRet, TruncSVD    

ps, pf, wf = buildWaterAndParkDicts(parkDir, waterDir)
pdf, ptsvd = yeildParkSVD(ps, pf, uniqueLocs, comps = 6)
wdf, wtsvd = yeildWaterSVD(wf, uniqueLocs, comps = 6)
pdf.columns = ['Park'+str(c) for c in pdf.columns]
wdf.columns = ['Water'+str(c) for c in wdf.columns]

<class 'numpy.float64'>
(80, 605)
TruncatedSVD(algorithm='randomized', n_components=6, n_iter=5,
       random_state=None, tol=0.0)
(80, 6)


In [158]:
wdf.head()

,Water0,Water1,Water2,Water3,Water4,Water5
"(-87.930995, 41.957799)",7.452501,-2.415331,2.122308,-0.353222,0.585042,0.305456
"(-87.890615, 41.974689)",6.846056,-2.593422,1.839218,-0.280489,0.525015,0.210701
"(-87.862995, 41.992478000000006)",6.625241,-2.652850,1.583163,-0.151807,0.425006,0.121209
"(-87.832763, 41.944869)",5.744749,-2.252962,1.059791,-0.233730,0.214307,0.021081
"(-87.824812, 41.974089)",5.919241,-2.453215,1.058257,-0.087411,0.206612,-0.006295


In [89]:
a=(1,2)
type(a) == tuple

True

In [77]:
def calculate_distances(loc, finder):
    Dist = {}
    for k in finder:
        Dist[k] = finder[k].query(loc,1)[0]

    return  Dist


def water_info(df, finder):
        uniqueLocs = df['Location'].unique()
        rows = {}
        for loc in uniqueLocs:
            rows[loc] = calculate_distances(loc,finder)
        toRet = pd.DataFrame(rows)
        toRet = toRet.transpose()
        toRet.index = [idx for idx in toRet.index]
        return toRet

In [78]:
tr = water_info(df, wf)
tr.head()

,0,1,2,3,4,5,6,7,8,9,...,595,596,597,598,599,600,601,602,603,604
"(-87.930995, 41.957799)",0.258189,0.273510,0.280411,0.283766,0.291122,0.291177,0.291197,0.291264,0.291516,0.291665,...,0.299097,0.292277,0.292162,0.292374,0.292514,0.292597,0.292728,0.292225,0.280558,0.288188
"(-87.890615, 41.974689)",0.231906,0.246757,0.252666,0.255866,0.262978,0.263047,0.263073,0.263157,0.263437,0.263598,...,0.269351,0.262915,0.262806,0.263074,0.263213,0.263353,0.263483,0.263216,0.252771,0.259516
"(-87.862995, 41.992478000000006)",0.221358,0.235498,0.240353,0.243353,0.250125,0.250207,0.250238,0.250337,0.250640,0.250810,...,0.254836,0.248866,0.248765,0.249085,0.249221,0.249415,0.249542,0.249511,0.240415,0.246194
"(-87.832763, 41.944869)",0.167559,0.182201,0.187844,0.191005,0.198062,0.198133,0.198160,0.198246,0.198531,0.198693,...,0.204270,0.197854,0.197745,0.198017,0.198156,0.198300,0.198429,0.198183,0.187939,0.194524
"(-87.824812, 41.974089)",0.181188,0.194910,0.199303,0.202214,0.208837,0.208923,0.208956,0.209059,0.209371,0.209543,...,0.213020,0.207193,0.207095,0.207429,0.207564,0.207773,0.207900,0.207941,0.199348,0.204757


In [71]:
t2 = tr
t2.index = t2.loc[:,0]
t2.head()

,0,1
0,,
"(-87.654224, 41.867108)","(-87.654224, 41.867108)","{0: 0.05121268778717819, 1: 0.0330911917091205..."
"(-87.64886, 41.862292)","(-87.64886, 41.862292)","{0: 0.054110365187564666, 1: 0.033676484353413..."
"(-87.655232, 41.896282)","(-87.655232, 41.896282)","{0: 0.06918619413580257, 1: 0.0580945279356783..."
"(-87.760886, 41.907645)","(-87.760886, 41.907645)","{0: 0.08898680037921794, 1: 0.102809727825564,..."
"(-87.666455, 41.9216)","(-87.666455, 41.9216)","{0: 0.0851739656124978, 1: 0.07992288387367903..."


In [135]:
wdf.head()

,Water0,Water1,Water2,Water3,Water4,Water5
"(-87.930995, 41.957799)",7.452501,-2.415331,2.122308,-0.353222,0.585042,0.305456
"(-87.890615, 41.974689)",6.846056,-2.593422,1.839218,-0.280489,0.525015,0.210701
"(-87.862995, 41.992478000000006)",6.625241,-2.652850,1.583163,-0.151807,0.425006,0.121209
"(-87.832763, 41.944869)",5.744749,-2.252962,1.059791,-0.233730,0.214307,0.021081
"(-87.824812, 41.974089)",5.919241,-2.453215,1.058257,-0.087411,0.206612,-0.006295


In [42]:
df_a = df_w.merge(pdf, how = 'outer', left_on='Location', right_index=True)
df_a = df_a.merge(wdf, how = 'outer', left_on = 'Location', right_index = True)

In [55]:
df_a.head(2)

,NumMosquitos,WnvPresent,Date,Species,Trap,Latitude,Longitude,Yr,Mo,Week,...,Park2,Park3,Park4,Park5,Water0,Water1,Water2,Water3,Water4,Water5
0,1,0,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,2007,5,22,...,29715.14289,67489.311127,61468.115233,3092.465154,3.82796,0.108809,-1.189728,-0.10479,0.184891,0.114866
1,2,0,2007-05-29,CULEX RESTUANS,T048,41.867108,-87.654224,2007,5,22,...,29715.14289,67489.311127,61468.115233,3092.465154,3.82796,0.108809,-1.189728,-0.10479,0.184891,0.114866


In [44]:
df_a.columns

Index(['NumMosquitos', 'WnvPresent', 'Date', 'Species', 'Trap', 'Latitude',
       'Longitude', 'Yr', 'Mo', 'Week', 'Location', 'CULEX PIPIENS',
       'CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX SALINARIUS',
       'CULEX TERRITANS', 'CULEX TARSALIS', 'CULEX ERRATICUS', 'DewPointamax',
       'DewPointamin', 'DewPointmean', 'StnPressureamax', 'StnPressureamin',
       'StnPressuremean', 'AvgSpeedamax', 'AvgSpeedamin', 'AvgSpeedmean',
       'temp_max', 'temp_min', 'temp_avg', 'temp_expected', 'temp_diff',
       'sunset', 'precip_total', 'precip_avg', 'Park0', 'Park1', 'Park2',
       'Park3', 'Park4', 'Park5', 'Water0', 'Water1', 'Water2', 'Water3',
       'Water4', 'Water5'],
      dtype='object')

### Spray


In [192]:
spray = pd.read_csv('./input/spray.csv')

spray['Date'] = pd.to_datetime(spray['Date'])

spray['Location'] = [(spray.loc[idx,'Longitude'], spray.loc[idx,'Latitude'])
                  for idx in spray.index]

spray = spray.drop(['Time'], axis = 'columns')

spray.shape

(14835, 4)

In [193]:
spray.head()

,Date,Latitude,Longitude,Location
0,2011-08-29,42.391623,-88.089163,"(-88.0891633333, 42.3916233333)"
1,2011-08-29,42.391348,-88.089163,"(-88.0891633333, 42.3913483333)"
2,2011-08-29,42.391022,-88.089157,"(-88.0891566667, 42.3910216667)"
3,2011-08-29,42.390637,-88.089158,"(-88.0891583333, 42.3906366667)"
4,2011-08-29,42.390410,-88.088858,"(-88.0888583333, 42.39041)"


In [206]:
sdu=spray.Date.unique()
sdu

array(['2011-08-29T00:00:00.000000000', '2011-09-07T00:00:00.000000000',
       '2013-07-17T00:00:00.000000000', '2013-07-25T00:00:00.000000000',
       '2013-08-08T00:00:00.000000000', '2013-08-15T00:00:00.000000000',
       '2013-08-16T00:00:00.000000000', '2013-08-22T00:00:00.000000000',
       '2013-08-29T00:00:00.000000000', '2013-09-05T00:00:00.000000000'], dtype='datetime64[ns]')

### Spray Categorical

In [198]:
list(spray[spray['Date'] ==sdu[0]]['Location'])

[(-88.089163333333303, 42.3916233333333),
 (-88.089163333333303, 42.391348333333298),
 (-88.089156666666696, 42.391021666666703),
 (-88.089158333333302, 42.390636666666701),
 (-88.088858333333306, 42.390409999999996),
 (-88.088315000000009, 42.390394999999998),
 (-88.088001666666699, 42.390673333333297),
 (-88.088001666666699, 42.391026666666704),
 (-88.088003333333305, 42.391403333333294),
 (-88.087994999999992, 42.391718333333294),
 (-88.0879883333333, 42.392038333333296),
 (-88.0880783333333, 42.392409999999998),
 (-88.088331666666704, 42.392800000000001),
 (-88.088369999999998, 42.393256666666701),
 (-88.088231666666701, 42.393599999999999),
 (-88.088416666666703, 42.393203333333297),
 (-88.088311666666698, 42.392791666666703),
 (-88.088210000000004, 42.392476666666703),
 (-88.088696666666692, 42.392384999999997),
 (-88.089113333333302, 42.392098333333294),
 (-88.089498333333296, 42.392179999999996),
 (-88.089790000000008, 42.392428333333299),
 (-88.090041666666693, 42.392721666666

In [200]:
data = df

In [234]:
spray_dict.keys()

dict_keys([datetime.date(2011, 8, 29), datetime.date(2011, 9, 7), datetime.date(2013, 7, 17), datetime.date(2013, 7, 25), datetime.date(2013, 8, 8), datetime.date(2013, 8, 15), datetime.date(2013, 8, 16), datetime.date(2013, 8, 22), datetime.date(2013, 8, 29), datetime.date(2013, 9, 5)])

In [245]:
close = []
for l in uniqueLocs:
    if spray_dict[pd.to_datetime(sdu[1]).date()].query(l,1)[0]*69<.1:
        close.append(l)
for l in close:
    print(l)
    td = data[data['Location']==l]['Date'].unique()
    for d in td:
        print(pd.Timedelta(d-sdu[1]))


(-87.795585000000003, 41.999128999999996)
-1562 days +00:00:00
-1555 days +00:00:00
-1534 days +00:00:00
-1531 days +00:00:00
-1519 days +00:00:00
-1512 days +00:00:00
-1503 days +00:00:00
-1498 days +00:00:00
-1492 days +00:00:00
-1484 days +00:00:00
-1478 days +00:00:00
-1475 days +00:00:00
-1464 days +00:00:00
-1450 days +00:00:00
-1444 days +00:00:00
-1434 days +00:00:00
(-87.811506000000008, 42.011600999999999)
-1555 days +00:00:00
-1534 days +00:00:00
-1531 days +00:00:00
-1528 days +00:00:00
-1519 days +00:00:00
-1512 days +00:00:00
-1503 days +00:00:00
-1498 days +00:00:00
-1492 days +00:00:00
-1484 days +00:00:00
-1478 days +00:00:00
-1475 days +00:00:00
-1464 days +00:00:00
-1450 days +00:00:00
-1444 days +00:00:00
-1434 days +00:00:00
-827 days +00:00:00
-817 days +00:00:00
-810 days +00:00:00
-803 days +00:00:00
-793 days +00:00:00
-789 days +00:00:00
-782 days +00:00:00
-775 days +00:00:00
-768 days +00:00:00
-761 days +00:00:00
-743 days +00:00:00
-741 days +00:00:00
-720

In [276]:
spray.groupby('Date')['Location'].count()

Date
2011-08-29      95
2011-09-07    2114
2013-07-17    2202
2013-07-25    1607
2013-08-08    1195
2013-08-15    2668
2013-08-16     141
2013-08-22    1587
2013-08-29    2302
2013-09-05     924
Name: Location, dtype: int64

In [275]:
spray['Date'].dt.date.unique()

array([datetime.date(2011, 8, 29), datetime.date(2011, 9, 7),
       datetime.date(2013, 7, 17), datetime.date(2013, 7, 25),
       datetime.date(2013, 8, 8), datetime.date(2013, 8, 15),
       datetime.date(2013, 8, 16), datetime.date(2013, 8, 22),
       datetime.date(2013, 8, 29), datetime.date(2013, 9, 5)], dtype=object)

In [278]:
spray_dict = {}

for d in spray['Date'].dt.date.unique():
    print(len(spray[spray['Date']==d]))
    spray_dict[d] = cKDTree( list(spray[spray['Date'] >= d ]['Location']) )

#def nearSpray(data, spray_dict, dist = .2):
toRet = []
dist = .5
spray_dates = list(spray_dict.keys())
spray_dates.sort(reverse = True)
firstSpray = spray_dates[-1]


for sDate in spray_dates:
    print(sDate)
    close = []
    for loc in data['Location'].unique():
        if spray_dict[sDate].query(loc,1)[0]*69 < dist:
            close.append(loc)
    print(len(close))
    for loc in close:
        td = data[data['Location'] == loc]['Date'].unique()
        for d in td:
            d = pd.to_datetime(d).date()
            
            if ((d - sDate < pd.Timedelta(0)) &
                    (d-sDate > pd.Timedelta(days=-8))):
                #print(d - sDate)
                toRet.append((loc, d, 'before'))
            if ((d - sDate > pd.Timedelta(0)) &
                    (d-sDate < pd.Timedelta(days=10))):
                toRet.append((loc, d, 'after'))

95
2114
2202
1607
1195
2668
141
1587
2302
924
2013-09-05
3
2013-08-29
9
2013-08-22
17
2013-08-16
19
2013-08-15
29
2013-08-08
32
2013-07-25
36
2013-07-17
45
2011-09-07
49
2011-08-29
49


In [288]:
df_w.columns

Index(['NumMosquitos', 'WnvPresent', 'Date', 'Species', 'Trap', 'Latitude',
       'Longitude', 'Yr', 'Mo', 'Week', 'Location', 'CULEX PIPIENS',
       'CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX SALINARIUS',
       'CULEX TERRITANS', 'CULEX TARSALIS', 'CULEX ERRATICUS', 'DewPointamax',
       'DewPointamin', 'DewPointmean', 'StnPressureamax', 'StnPressureamin',
       'StnPressuremean', 'AvgSpeedamax', 'AvgSpeedamin', 'AvgSpeedmean',
       'temp_max', 'temp_min', 'temp_avg', 'temp_expected', 'temp_diff',
       'sunset', 'precip_total', 'precip_avg'],
      dtype='object')

In [302]:

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8475 entries, 0 to 8474
Data columns (total 36 columns):
NumMosquitos              8475 non-null int64
WnvPresent                8475 non-null int32
Date                      8475 non-null object
Species                   8475 non-null object
Trap                      8475 non-null object
Latitude                  8475 non-null float64
Longitude                 8475 non-null float64
Yr                        8475 non-null int64
Mo                        8475 non-null int64
Week                      8475 non-null int64
Location                  8475 non-null object
CULEX PIPIENS             8475 non-null int32
CULEX PIPIENS/RESTUANS    8475 non-null int32
CULEX RESTUANS            8475 non-null int32
CULEX SALINARIUS          8475 non-null int32
CULEX TERRITANS           8475 non-null int32
CULEX TARSALIS            8475 non-null int32
CULEX ERRATICUS           8475 non-null int32
DewPointamax              8475 non-null int64
DewPointami

In [306]:
df2=df_w
df2['Date'] = [pd.to_datetime(d) for d in df2['Date']]
df2 = df2.merge(tr, left_on = ['Location', 'Date'], right_on = ['Location','Date'], how = 'left')
df2[(df2['Location'] == tr.loc[0,'Location']) & (df2['Date'] == tr.loc[0,'Date'])]
df2.Spray.describe()

count       594
unique        2
top       after
freq        339
Name: Spray, dtype: object

In [307]:
import pickle
with open('sdf.pickle', 'wb') as f:
    pickle.dump(df2, f, protocol = 0)

In [305]:
tr =pd.DataFrame(toRet)
tr.columns = ['Location','Date','Spray']
tr['Date'] = [pd.to_datetime(d) for d in tr['Date']]
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 3 columns):
Location    297 non-null object
Date        297 non-null datetime64[ns]
Spray       297 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 7.0+ KB


In [279]:
spray_dict = {}

for d in spray['Date'].dt.date.unique():
    spray_dict[d] = cKDTree( list(spray[spray['Date'] == d ]['Location']) )

def nearSpray(data, spray_dict, dist = .2):
    toRet = {}

    spray_dates = list(spray_dict.keys())
    spray_dates.sort(reverse = True)
    firstSpray = spray_dates[-1]

    for l in data['Location'].unique():
        toRet[l] = {}
        trapDates = data[ data['Location'] ==l]['Date'].unique()

        trapDates = [pd.Timestamp(d).date() for d in trapDates]
        trapDates.sort()

        if trapDates[-1] > firstSpray:
            for tDate in trapDates:
                before, after = False, False
                for sDate in spray_dates:



                    if ((tDate - sDate < pd.Timedelta(0)) &
                    (tDate-sDate > pd.Timedelta(days=-10))):
                        #print('yes')
                        #print(spray_dict[sDate].query(l,1)[0]*69)
                        if (spray_dict[sDate].query(l,1)[0]*69 < dist):
                            #print('yes')
                            before = True

                    if ((tDate - sDate > pd.Timedelta(0)) &
                    (tDate-sDate < pd.Timedelta(days=10)) & 
                   (spray_dict[sDate].query(l,1)[0]*69 < dist)):
                        #print('hi')
                        after = True

                toRet[l][tDate] = (before, after)

                toRet2 = {}
                for k1 in toRet:
                    for k2 in toRet[k1]:
                        toRet2[(k1,k2)] = toRet[k1][k2]

    df = pd.DataFrame().from_dict(toRet2).transpose()
    df.columns = ['SprayBefore', 'SprayAfter']
    df['Location'] = df.index.map(lambda x: x[0])
    df['Date'] = df.index.map(lambda x: x[1])
    df.index = range(0,len(df))
                
    return df
nearspray = nearSpray(df, spray_dict)
nearspray.shape

(3847, 4)

In [280]:
nearspray['SprayAfter'].sum()

22

In [214]:
uniqueLocs = data['Location'].unique()
ul = uniqueLocs[0]
ul

(-87.654223999999999, 41.867108000000002)

In [216]:
trapDates = data[ data['Location'] ==ul]['Date'].unique()

In [220]:

trapDates = [pd.Timestamp(d).date() for d in trapDates]
trapDates.sort()
trapDates[-1]> firstSpray

True

array(['2011-08-29T00:00:00.000000000', '2011-09-07T00:00:00.000000000',
       '2013-07-17T00:00:00.000000000', '2013-07-25T00:00:00.000000000',
       '2013-08-08T00:00:00.000000000', '2013-08-15T00:00:00.000000000',
       '2013-08-16T00:00:00.000000000', '2013-08-22T00:00:00.000000000',
       '2013-08-29T00:00:00.000000000', '2013-09-05T00:00:00.000000000'], dtype='datetime64[ns]')

In [226]:
for d in sdu:
    print(pd.to_datetime(trapDates[-1]) - pd.to_datetime(d)> pd.Timedelta(days = 22))

True
True
True
True
True
True
True
True
True
False


In [211]:
df['SprayBefore'].sum()

9

In [205]:
firstSpray

datetime.date(2011, 8, 29)

In [204]:
data.head(2)

,NumMosquitos,WnvPresent,Date,Species,Trap,Latitude,Longitude,Yr,Mo,Week,Location,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TERRITANS,CULEX TARSALIS,CULEX ERRATICUS
0,1,0,2007-05-29,CULEX PIPIENS/RESTUANS,T048,41.867108,-87.654224,2007,5,22,"(-87.654224, 41.867108)",0,1,0,0,0,0,0
1,2,0,2007-05-29,CULEX RESTUANS,T048,41.867108,-87.654224,2007,5,22,"(-87.654224, 41.867108)",0,0,1,0,0,0,0


In [178]:
nearspray.head(10)

,SprayBefore,SprayAfter,Loc,Date
0,False,False,"(-87.930995, 41.957799)",2009-06-03
1,False,False,"(-87.930995, 41.957799)",2009-06-05
2,False,False,"(-87.930995, 41.957799)",2009-06-15
3,False,False,"(-87.930995, 41.957799)",2009-06-22
4,False,False,"(-87.930995, 41.957799)",2009-06-29
5,False,False,"(-87.930995, 41.957799)",2009-07-13
6,False,False,"(-87.930995, 41.957799)",2009-07-17
7,False,False,"(-87.930995, 41.957799)",2009-07-27
8,False,False,"(-87.930995, 41.957799)",2009-07-31
9,False,False,"(-87.930995, 41.957799)",2009-08-07


In [163]:
mask = (df_w['Date'].dt.year == 2011) | (df_w['Date'].dt.year ==2013)
df_s = df_w.loc[mask]

#df_s = df_s.merge(nearspray)

In [179]:
nearspray.SprayBefore.sum()

19

In [162]:
df_w.shape

(8475, 35)

In [171]:
df_s = df_s.merge(nearspray, left_on = ['Location','Date'], right_on = ['Loc','Date'])

In [175]:
df_s['SprayAfter'].sum()

23

In [164]:
df_s.shape

(3785, 35)

In [191]:
df_a.Location

0       (-87.654224, 41.867108)
1       (-87.654224, 41.867108)
25      (-87.654224, 41.867108)
26      (-87.654224, 41.867108)
27      (-87.654224, 41.867108)
87      (-87.654224, 41.867108)
204     (-87.654224, 41.867108)
284     (-87.654224, 41.867108)
285     (-87.654224, 41.867108)
417     (-87.654224, 41.867108)
418     (-87.654224, 41.867108)
578     (-87.654224, 41.867108)
579     (-87.654224, 41.867108)
671     (-87.654224, 41.867108)
672     (-87.654224, 41.867108)
1016    (-87.654224, 41.867108)
1017    (-87.654224, 41.867108)
1228    (-87.654224, 41.867108)
1229    (-87.654224, 41.867108)
1230    (-87.654224, 41.867108)
1484    (-87.654224, 41.867108)
1485    (-87.654224, 41.867108)
1727    (-87.654224, 41.867108)
1728    (-87.654224, 41.867108)
2101    (-87.654224, 41.867108)
2220    (-87.654224, 41.867108)
2221    (-87.654224, 41.867108)
2415    (-87.654224, 41.867108)
2416    (-87.654224, 41.867108)
2584    (-87.654224, 41.867108)
                 ...           
8002    

In [54]:
allDF = [df_w, df_a, df_s]
with open('allDF.pickle', 'wb') as f:
    pickle.dump(allDF, f, protocol = 0)